# Taller Práctico 3: Construcción de Modelos Analíticos de Ventas Escalonadas y Venta Cruzada

**Objetivos del Taller:**

* Comprender la importancia de las estrategias de venta escalonada y venta cruzada en el aumento de ingresos y fidelización de clientes.

* Aprender a identificar patrones en los datos que indiquen oportunidades para implementar estas estrategias.

* Desarrollar modelos analíticos que permitan optimizar y personalizar las recomendaciones de productos.

# Construcción modelos analíticos de Ventas Escalonadas y Ventas Cruzadas

## Instalar las bibliotecas necesarias

Antes de comenzar con el análisis, debemos asegurarnos de tener todas las bibliotecas necesarias instaladas y cargadas en nuestro entorno de trabajo. Estas bibliotecas proporcionan herramientas fundamentales para manipular datos, realizar análisis y construir modelos que nos permitan explorar y optimizar estrategias como la venta cruzada y venta escalonada. A continuación, se mencionan las bibliotecas que utilizaremos en este notebook:

* **numpy:** Para realizar operaciones matemáticas avanzadas y manipulación eficiente de arreglos numéricos.
* **pandas:** Herramienta clave para el análisis y manipulación de datos en formato tabular.
* **matplotlib:** Para la creación de visualizaciones personalizadas y claras.
* **seaborn:** Biblioteca para generar gráficos estadísticos más atractivos y detallados.
* **sklearn:** Conjunto de herramientas para machine learning, utilizadas en tareas de clasificación, regresión y más.
* **mlxtend:** Proporciona herramientas avanzadas para análisis de patrones frecuentes y generación de reglas de asociación, muy útiles en el análisis de estrategias de ventas.

In [ ]:
!pip install mlxtend

## Cargar las bibliotecas previamente instaladas

Ahora que tenemos instaladas todas las bibliotecas necesarias, procedemos a cargarlas en nuestro entorno de trabajo.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from mlxtend.frequent_patterns import apriori, association_rules

## Cargar el conjunto de datos

Con las bibliotecas listas, el siguiente paso es cargar nuestro conjunto de datos que utilizaremos para la construcción de los modelos analíticos de ventas cruzadas y escalonadas.

In [ ]:
# Carga y descripción de los datos
data = pd.read_csv("./data/customer_transactions_dataset.csv")
data

## Entendimiento de los Datos

Es fundamental entender la estructura y las características del conjunto de datos. A continuación, exploraremos el conjunto de datos utilizando a través de las visualizaciones de las principales variables.

In [ ]:
# Gasto promedio por edad
avg_spending_per_age = data.groupby('Segmento de Mercado')['Valor'].mean()
avg_spending_per_age.plot(kind='line', figsize=(8, 5), marker='o', color='orange')
plt.title('Gasto Promedio por Edad')
plt.xlabel('Edad')
plt.ylabel('Gasto Promedio')
plt.tight_layout()
plt.show()

In [ ]:
# Asegurarse de que la columna 'Fecha de Compra' sea de tipo datetime
data['Fecha de Compra'] = pd.to_datetime(data['Fecha de Compra'])

# Frecuencia de compra (días entre compras) por cliente
freq_purchase = data.groupby('ID Cliente')['Fecha de Compra'].apply(
    lambda x: x.sort_values().diff().mean().days
)

freq_purchase.hist(bins=20, figsize=(8, 5))
plt.title('Frecuencia de Compra (Promedio de Días entre Compras)')
plt.xlabel('Días')
plt.ylabel('Cantidad de Clientes')
plt.tight_layout()
plt.show()

## Preparación de los Datos

Es esencial preparar el conjunto de datos. Esto implica limpiar y transformar los datos para que sean coherentes y adecuados para los algoritmos de machine learning. A continuación, realizamos la eliminación de duplicados y el manejo de valores nulos.

### Eliminación de registros duplicados

El primer paso que haremos es eliminar cualquier duplicado en el conjunto de datos, en este contexto quiere decir, eliminar aquellas transacciones que se registraron más de una vez por error. Los duplicados pueden afectar la precisión del modelo y distorsionar las métricas calculadas, como las frecuencias y promedios.

In [ ]:
# Eliminar registros duplicados en el conjunto de datos
data = data.drop_duplicates()
data.head()

### Eliminación de registros con valores nulos

A continuación, eliminamos las filas con valores nulos. Esto asegura que las métricas calculadas no se vean afectadas por datos incompletos. Es importante evaluar previamente si la eliminación de estas filas afecta significativamente el tamaño de la muestra.

In [ ]:
# Eliminar filas con valores nulos en el conjunto de datos
data = data.dropna()
data.head()

# Introducción a la Venta Cruzada (Cross-Selling)

La venta cruzada es una estrategia comercial que busca maximizar el valor de cada transacción ofreciendo productos o servicios complementarios al cliente durante el proceso de compra. Esta técnica, utilizada en una amplia variedad de industrias, permite a las empresas satisfacer múltiples necesidades del cliente, aumentando así su nivel de satisfacción y los ingresos totales.

A continuación, se detallan algunos beneficios clave de la venta cruzada:

* **Incremento del Valor del Cliente:** Al ofrecer productos relacionados, se maximiza el valor promedio de cada cliente durante su ciclo de vida, lo que contribuye al crecimiento sostenible del negocio.

* **Fortalecimiento de la Relación con el Cliente:** Al identificar y atender necesidades adicionales, se crea una percepción de atención personalizada, fomentando la fidelidad y satisfacción del cliente.

* **Optimización del Portafolio de Productos:** La venta cruzada permite dar visibilidad a productos complementarios que podrían pasar desapercibidos, aumentando su rotación y promoviendo un uso más eficiente del inventario.

* **Aumento del Ticket Promedio:** Con una estrategia bien ejecutada, las empresas logran incrementar el monto promedio de cada transacción sin necesidad de atraer nuevos clientes.

La venta cruzada no solo mejora los resultados financieros de la empresa, sino que también refuerza su capacidad para ofrecer soluciones integrales a sus clientes. Cuando se implementa correctamente, esta estrategia crea una experiencia de compra más satisfactoria y completa.

## Construcción de la Matriz de Cesta

Para implementar estrategias de venta cruzada, es fundamental entender qué productos suelen comprarse juntos. Esto se logra analizando el comportamiento de compra de los clientes mediante una **matriz de cesta**. Esta matriz organiza los datos de tal manera que cada fila representa a un cliente y cada columna corresponde a un producto. Los valores en la matriz reflejan la cantidad de unidades compradas de cada producto por cliente.

#### Proceso para construir la matriz de cesta:
1. **Agrupar los datos:** Los datos se agrupan por cliente y producto, lo que permite consolidar la información sobre la cantidad de productos adquiridos por cliente.
2. **Pivotear los datos:** Se utiliza una estructura de tabla pivotante para transformar las filas en columnas, donde cada columna es un producto.
3. **Rellenar valores nulos:** Los valores faltantes (que representan productos no comprados) se rellenan con ceros para facilitar el análisis.

A continuación, generamos la matriz de cesta a partir de los datos agrupados y estructurados.

In [ ]:
# Generar la matriz de cesta (basket matrix)
basket = data.groupby(['ID Cliente', 'Nombre del Producto'])['Cantidad'].sum().unstack().fillna(0)
basket

Una vez creada la matriz de cesta, es necesario convertir los valores numéricos en valores booleanos. Esto se debe a que el análisis de patrones frecuentes y reglas de asociación, típicos en estrategias de venta cruzada, requiere identificar únicamente si un producto fue comprado o no, sin considerar la cantidad.

#### Conversión a valores booleanos:
- Los valores mayores a cero se convierten en `True`, indicando que el cliente compró ese producto.
- Los valores iguales a cero permanecen como `False`, señal

In [ ]:
basket = basket.applymap(lambda x: True if x > 0 else False)
basket

## Generación de Reglas de Asociación (APRIORI)

Con la matriz de cesta preparada, el siguiente paso es identificar patrones frecuentes de compra y generar reglas de asociación. Este análisis es crucial para implementar estrategias efectivas de venta cruzada, ya que permite descubrir qué productos se compran juntos con mayor frecuencia y qué relaciones existen entre ellos.

#### Proceso para generar las reglas:

1. **Aplicar el algoritmo Apriori:** 

   - Identifica conjuntos de productos frecuentes basándose en un umbral de soporte mínimo (`min_support`).
   
   - Esto significa que solo se consideran combinaciones de productos que aparecen en un porcentaje significativo de las transacciones.

2. **Generar reglas de asociación:**

   - Utilizamos métricas como `lift`, `confidence` y `support` para evaluar la fuerza y relevancia de las reglas.
   
   - Estas métricas ayudan a priorizar las combinaciones de productos más útiles para la venta cruzada.


3. **Seleccionar las reglas más relevantes:** 

   - Filtramos las reglas con mayor lift y confidence para identificar aquellas que son más impactantes y confiables en el contexto del negocio.

A continuación, mostramos las reglas de asociación más relevantes, incluyendo los productos involucrados (antecedentes y consecuentes) y las métricas clave.

In [ ]:
# Aplicar el algoritmo Apriori
frequent_items = apriori(basket, min_support=0.01, use_colnames=True)

# Generar reglas de asociación
rules = association_rules(frequent_items, metric="lift", min_threshold=2.0, num_itemsets=len(frequent_items))

# Mostrar las reglas más relevantes
top_rules = rules.head(15)

# Mostrar reglas seleccionadas
top_rules[['antecedents', 'consequents', 'lift', 'confidence', 'support']]

* **Support (Soporte)**: Es la proporción de transacciones que contienen un conjunto de ítems respecto al total. Nos ayuda a identificar qué combinaciones de productos son frecuentes en el total de ventas, mostrando patrones relevantes.

* **Confidence (Confianza)**: Es la probabilidad de que, dado que un cliente compra el conjunto X, también compre el conjunto Y. Ayuda a entender qué productos suelen venderse juntos, clave para diseñar promociones efectivas.

* **Lift:** Mide qué tan dependiente es la relación entre X e Y comparada con una relación aleatoria. Un lift > 1 indica que la compra de X incrementa la probabilidad de comprar Y, lo cual es crucial para detectar oportunidades en ventas cruzadas o escalonadas.

### Visualización de las Reglas con un Mapa de Calor

Para interpretar mejor las reglas de asociación y facilitar la identificación de relaciones significativas entre productos, creamos un **mapa de calor** basado en los valores de `lift`. Esta visualización permite observar de manera clara la fortaleza de las relaciones entre los productos (antecedentes y consecuentes) generadas por el algoritmo Apriori.

In [ ]:
# Crear una matriz para el mapa de calor
top_rules_pivot = top_rules.pivot(index='antecedents', columns='consequents', values='lift').fillna(0)

# Mapa de calor
plt.figure(figsize=(12, 8))
sns.heatmap(top_rules_pivot, annot=True, fmt=".2f", cmap="YlGnBu", cbar_kws={'label': 'Lift'})
plt.title('Mapa de Calor: Lift de las Reglas Principales')
plt.xlabel('Consequents')
plt.ylabel('Antecedents')
plt.show()

### Análisis del Lift Promedio por Antecedentes

Para profundizar aún más en las relaciones descubiertas, calculamos el **lift promedio** para cada conjunto de productos antecedentes. Esto nos permite identificar cuáles son los productos que, al ser comprados, tienen mayor capacidad de influir en la compra de otros productos.

In [ ]:
# Agrupar por 'antecedents' y calcular el lift promedio
average_lift = top_rules.groupby('antecedents')['lift'].mean().reset_index()

# Convertir 'antecedents' de frozenset a string para facilitar la visualización
average_lift['antecedents'] = average_lift['antecedents'].apply(lambda x: ', '.join(list(x)))

# Ordenar los antecedentes por lift promedio para una mejor visualización
average_lift = average_lift.sort_values(by='lift', ascending=False)

# Graficar las reglas principales por lift promedio
plt.figure(figsize=(10, 6))
plt.bar(average_lift['antecedents'], average_lift['lift'], color='skyblue')
plt.title('Lift Promedio de las Reglas Principales por Antecedents')
plt.xlabel('Antecedents')
plt.ylabel('Lift Promedio')
plt.xticks(rotation=90)
plt.grid(axis='y', linestyle='--')
plt.show()

# Introducción a la Venta Escalonada (Up-Selling)

La venta escalonada es una estrategia comercial que busca maximizar el valor de cada transacción persuadiendo al cliente para que adquiera una versión más avanzada, completa o costosa del producto o servicio que ya está considerando. A diferencia de la venta cruzada, la venta escalonada se enfoca en mejorar la calidad o las características de la compra inicial, aumentando el ticket promedio y fortaleciendo la percepción de valor del cliente.

A continuación, se detallan algunos beneficios clave de la venta escalonada:

* **Aumento del Ticket Promedio:** Al incentivar la compra de versiones premium o productos con características adicionales, se logra incrementar el monto promedio por transacción de manera significativa.

* **Fortalecimiento de la Percepción de Valor:** La venta escalonada ayuda a los clientes a visualizar los beneficios adicionales de una opción más avanzada, lo que refuerza su decisión de inversión y eleva la satisfacción del cliente.

* **Optimización de la Experiencia del Cliente:** Al ofrecer soluciones más completas, se atienden mejor las necesidades y expectativas del cliente, creando una experiencia de compra superior.

* **Fidelización a través de Beneficios Exclusivos:** La oferta de productos escalonados, como servicios VIP o características adicionales, genera una percepción de exclusividad que fortalece la relación con el cliente.

La venta escalonada no solo impacta positivamente en los ingresos, sino que también refuerza la relación con el cliente al presentarle opciones mejoradas que se alinean con sus necesidades y aspiraciones. Implementada correctamente, esta estrategia permite a las empresas posicionar su oferta premium como la elección lógica para maximizar el valor percibido.

## Construcción de un Modelo Predictivo para Venta Escalonada

En el contexto de la venta escalonada, la identificación de clientes con mayor probabilidad de aceptar una oferta de un producto o servicio de mayor valor es esencial para maximizar el éxito de esta estrategia. Para lograrlo, estamos desarrollando un modelo predictivo basado en aprendizaje supervisado que nos permita anticipar estos comportamientos.

#### Proceso para la construcción del modelo:

1. **Preparación de los datos:**
   - **Codificación de variables categóricas:** Convertimos variables como el canal de venta o segmento de mercado en representaciones numéricas mediante técnicas como `One-Hot Encoding`.
   - **Definición de la variable objetivo:** Creamos una etiqueta binaria (`y`) que indica si una transacción tiene un valor alto (mayor al percentil 50 o 75, según el criterio definido). Esto nos ayuda a identificar patrones relacionados con compras de mayor valor.

2. **División de los datos:**
   - Separar el conjunto de datos en entrenamiento y prueba asegura una evaluación objetiva del modelo. Utilizamos un 70% de los datos para el entrenamiento y un 30% para las pruebas.

3. **Construcción del modelo:**
   - Seleccionamos un modelo de clasificación supervisado, como el **Random Forest**, que es robusto, eficiente y capaz de manejar relaciones no lineales entre las características.
   - Entrenamos el modelo con los datos de entrenamiento y lo utilizamos para predecir la probabilidad de que un cliente acepte una oferta escalonada.

In [ ]:
# Codificación de variables categóricas
data_encoded = pd.get_dummies(data, columns=['Canal de Venta'])

In [ ]:
# Dividir datos en entrenamiento y prueba
X = data_encoded.drop(['Valor', 'Nombre del Cliente', 'ID Cliente', 'ID Producto', 'Fecha de Compra', 'Nombre del Producto'], axis=1)
y = data_encoded['Valor'] > data_encoded['Valor'].median()*1.5  # Etiqueta binaria: alto valor de transacción
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Modelo de clasificación
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

### Evaluación del Modelo

Una vez entrenado el modelo de clasificación, evaluamos su desempeño para determinar qué tan bien predice la probabilidad de que un cliente acepte una oferta escalonada. Para ello, utilizamos métricas clave de clasificación que permiten analizar la precisión y efectividad del modelo.

#### Métricas principales:
- **Precision:** Indica qué porcentaje de las predicciones positivas son correctas. Es crucial para minimizar ofertas innecesarias a clientes que no aceptarían una opción escalonada.
- **Recall (Sensibilidad):** Mide la capacidad del modelo para identificar correctamente a los clientes que realmente aceptarían la oferta escalonada.
- **F1-Score:** Proporciona un balance entre precision y recall, siendo especialmente útil en casos con distribución desigual de clases.
- **Soporte:** Muestra la cantidad de observaciones verdaderas en cada clase, lo que da contexto al resto de las métricas.

In [ ]:
print(classification_report(y_test, y_pred))

### Visualización de la Matriz de Confusión

Para complementar la evaluación del modelo, utilizamos una **matriz de confusión**, que ofrece una representación visual de los resultados de las predicciones del modelo. Esto permite identificar patrones de errores y analizar cómo el modelo clasifica las diferentes clases.

In [ ]:
# Calcular la matriz de confusión
cm = confusion_matrix(y_test, y_pred)

# Visualizar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["No Up-Sell", "Up-Sell"])
disp.plot(cmap="Blues")
plt.title("Matriz de Confusión")
plt.show()

### Importancia de las Variables en el Modelo

Para entender qué características tienen mayor impacto en las predicciones del modelo, analizamos la **importancia de las variables**. Este análisis ayuda a identificar qué factores contribuyen más a la probabilidad de que un cliente acepte una oferta escalonada, proporcionando información valiosa para ajustar estrategias comerciales y focalizar esfuerzos.

#### Pasos realizados:
1. **Obtener la importancia de las variables:**

   - Utilizamos el atributo `feature_importances_` del modelo Random Forest para calcular la contribución relativa de cada característica al desempeño del modelo.

2. **Visualizar la importancia:**

   - Creamos un gráfico de barras horizontales que muestra la importancia de cada variable, permitiendo identificar rápidamente las características más relevantes.

In [ ]:
importances = model.feature_importances_
features = X_train.columns

# Graficar importancias de las variables
plt.figure(figsize=(10, 6))
plt.barh(features, importances, color="skyblue")
plt.title("Importancia de las Variables")
plt.xlabel("Importancia")
plt.ylabel("Variable")
plt.grid(axis="x", linestyle="--")
plt.show()